In [4]:
import torch
import numpy as np

# Create a small example with 2 users and 3 items
# edges: u1->i1, u1->i2, u2->i2

# First create edge indices
edge_index = torch.tensor([
    [0, 0, 1],    # user indices (u1,u1,u2)
    [2, 3, 3]     # item indices (i1,i2,i2)
])

# Calculate degrees for each node
unique_nodes, degrees = torch.unique(edge_index, return_counts=True)
degree_dict = dict(zip(unique_nodes.tolist(), degrees.tolist()))

# Calculate normalized edge weights
edge_weights = []
for src, dst in zip(edge_index[0], edge_index[1]):
    src_deg = degree_dict[src.item()]
    dst_deg = degree_dict[dst.item()]
    weight = 1 / np.sqrt(src_deg * dst_deg)
    edge_weights.append(weight)

# Make sure edge_weights is float32
edge_weights = torch.tensor(edge_weights, dtype=torch.float32)

# Create embeddings (2 users + 3 items = 5 nodes total)
embedding_dim = 2
embeddings = torch.tensor([
    [1.0, 2.0],  # u1
    [3.0, 4.0],  # u2
    [5.0, 6.0],  # i1
    [7.0, 8.0],  # i2
    [9.0, 10.0]  # i3
], dtype=torch.float32)  # Explicitly set to float32

print("Initial embeddings:")
print(embeddings)

# Create indices and values for sparse matrix
n_nodes = 5
indices = edge_index
values = edge_weights

# Create sparse matrix with float32
sparse_adj = torch.sparse_coo_tensor(indices, values, (n_nodes, n_nodes), dtype=torch.float32)

# Convert to dense for visualization
dense_adj = sparse_adj.to_dense()
print("\nNormalized adjacency matrix:")
print(dense_adj)

# Do matrix multiplication
result_dense = torch.mm(dense_adj, embeddings)
print("\nResult after message passing (matrix multiplication):")
print(result_dense)

# Let's show explicitly how u1's embedding was computed
u1_neighbors = edge_index[1][edge_index[0] == 0]  # i1, i2
u1_weights = edge_weights[edge_index[0] == 0]

print("\nManual computation for u1:")
print(f"u1 connects to items {u1_neighbors.tolist()} with weights {u1_weights.tolist()}")
manual_result = torch.zeros(embedding_dim, dtype=torch.float32)
for neighbor, weight in zip(u1_neighbors, u1_weights):
    neighbor_emb = embeddings[neighbor]
    print(f"Adding: {weight:.4f} * {neighbor_emb} = {weight * neighbor_emb}")
    manual_result += weight * neighbor_emb

print("\nManual result for u1:", manual_result)
print("Matches matrix result:", torch.allclose(manual_result, result_dense[0]))

Initial embeddings:
tensor([[ 1.,  2.],
        [ 3.,  4.],
        [ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

Normalized adjacency matrix:
tensor([[0.0000, 0.0000, 0.7071, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7071, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

Result after message passing:
tensor([[7.0355, 8.2426],
        [4.9497, 5.6569],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]])

Manual computation for u1:
u1 connects to items [2, 3] with weights [0.7071067690849304, 0.5]
Adding: 0.7071 * tensor([5., 6.]) = tensor([3.5355, 4.2426])
Adding: 0.5000 * tensor([7., 8.]) = tensor([3.5000, 4.0000])

Manual result for u1: tensor([7.0355, 8.2426])
Matches matrix result: True


In [5]:
0.7071*5+0.5*7

7.0355